<a href="https://colab.research.google.com/github/ETaktashova/Git-commands/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22trylangchain_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(пока проверены текстовый файл и эксель)

In [ ]:
!pip install llama-cpp-python chromadb langchain sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 31.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 M

In [ ]:
import warnings
from pprint import pprint as pp
from pprint import pp
from langchain.chains import LLMChain
from langchain.document_loaders.text import TextLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.document_loaders.directory import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.embeddings.llamacpp import LlamaCppEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from sentence_transformers import SentenceTransformer
from langchain_community.llms import GPT4All


In [ ]:
!pip install unstructured
# для экселя

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=2a369cb8bdad14034a5289256bd362f5c8b6e3ac8eb5cc5832aaed1cad06307e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
# для текста скачивание и чанкование, обернуть в функцию
path = ''
text_loader_kwargs = {'autodetect_encoding': True}
text_loader = DirectoryLoader(
    path,
    glob="**/*info.txt",
    loader_cls=TextLoader,
    loader_kwargs=text_loader_kwargs
)
t_docs = text_loader.load()
print(t_docs[0])
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
)
t_splitted_data = splitter.split_documents(t_docs)
pp(f'first chunk - {t_splitted_data[0].page_content}')
print(f'len of splitted data - {len(t_splitted_data)}')

page_content='Принципы здорового питания: это должен знать каждый!\n\nС чего же начать? Прежде всего, с твердого решения питаться правильно.\n Вот несколько несложных правил, на которые опирается практически вся современная диетология и \n которые помогут быстро освоить принципы здорового питания.\n    Правило № 1. Навсегда забудьте про фастфуд и постарайтесь не \n    злоупотреблять сладким.\n    Правило № 2. Ешьте как можно больше сезонных продуктов — в них \n    содержится максимум полезных веществ. Зимние овощи и фрукты, выращенные в теплицах \n    или привезенные издалека, из-за обработки химикатами и длительного хранения не только \n    теряют всю пользу, но и становятся аккумуляторами нитратов и других вредных химических соединений.\n    Правило № 3. Максимально ограничьте употребление рафинированных продуктов: сахара, растительного масла,\n    белой пшеничной муки, очищенного белого риса. В них нет клетчатки, которая очень важна для работы \n    пищеварительного тракта, а также 

In [ ]:
# для экселя скачивание и чанкование, обернуть в функц
path = ''
xl_loader = UnstructuredExcelLoader(
    "data3.xlsx",
    mode="elements"
)
x_docs = xl_loader.load()
print(x_docs[0])

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False
)
x_splitted_data = splitter.split_documents(x_docs)
pp(f'first chunk - {x_splitted_data[0].page_content}')
print(f'len of splitted data - {len(x_splitted_data)}')

In [ ]:
# https://www.sbert.net/docs/pretrained_models.html варианты моделей
# var_of_model = "all-MiniLM-L12-v2" - неплохо
import sentence_transformers
from langchain_community.vectorstores.utils import filter_complex_metadata

var_of_model = 'multi-qa-MiniLM-L6-cos-v1'
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
persist_directory = './chroma_db'
embedding = SentenceTransformerEmbeddings(model_name=var_of_model)
# Создание вект хранилища на диске
# фильтр для эксель
filter_xls = filter_complex_metadata(x_splitted_data)
# db for excel
# vectordb = Chroma.from_documents(
#     x_splitted_data,
#     embedding
# )
# db for txt
vectordb = Chroma.from_documents(
    t_splitted_data,
    embedding
)
print(f'collection count in vectordb in llm = {vectordb._collection.count()}')
vectordb.persist()
print(vectordb._collection.get())

collection count in vectordb in llm = 25
{'ids': ['71aa6796-bb62-11ee-a675-0242ac1c000c', '71aa6890-bb62-11ee-a675-0242ac1c000c', '71aa6908-bb62-11ee-a675-0242ac1c000c', '71aa6976-bb62-11ee-a675-0242ac1c000c', '71aa69e4-bb62-11ee-a675-0242ac1c000c', '71aa6a3e-bb62-11ee-a675-0242ac1c000c', '71aa6aa2-bb62-11ee-a675-0242ac1c000c', '71aa6b06-bb62-11ee-a675-0242ac1c000c', '71aa6b60-bb62-11ee-a675-0242ac1c000c', '71aa6bc4-bb62-11ee-a675-0242ac1c000c', '71aa6c1e-bb62-11ee-a675-0242ac1c000c', '71aa6c82-bb62-11ee-a675-0242ac1c000c', '71aa6cdc-bb62-11ee-a675-0242ac1c000c', '71aa6d36-bb62-11ee-a675-0242ac1c000c', '71aa6da4-bb62-11ee-a675-0242ac1c000c', '71aa6e08-bb62-11ee-a675-0242ac1c000c', '71aa6e62-bb62-11ee-a675-0242ac1c000c', '71aa6ebc-bb62-11ee-a675-0242ac1c000c', '71aa6f2a-bb62-11ee-a675-0242ac1c000c', '71aa6fac-bb62-11ee-a675-0242ac1c000c', '71aa7010-bb62-11ee-a675-0242ac1c000c', '71aa706a-bb62-11ee-a675-0242ac1c000c', '71aa70c4-bb62-11ee-a675-0242ac1c000c', '71aa711e-bb62-11ee-a675-0242a

In [ ]:
# для удаления старой бд после использования загруженных в нее данных
vectordb.delete_collection()

In [ ]:
# для примера поиска по бд
# ? только последний чанк всегда подходит
# question = "почему газировки лучше не пить"
question = "какие использовать растительные масла"
# question = 'сколько стоили макароны в ямало-ненецком ао?'
embedded_question = embedding.embed_query(question)
# answers = vectordb.similarity_search_by_vector(embedded_question, k=2)
answers = vectordb.similarity_search(question, k=3)
def printAnswer(answers):
    print(f'Наиболее релевантными для вопроса "{question}" являются чанки: ')
    print()
    for answer in answers:
        print(answer.page_content)
        # print(answer.metadata)  # .source
        print('-----')
printAnswer(answers)

# нет разницы в результатах между similarity_search
# и similarity_search_by_vector

Наиболее релевантными для вопроса "какие использовать растительные масла" являются чанки: 

недоваренной и заправляться растительным маслом. Мясные и рыбные продукты обязательно должны быть термически 
    обработаны, поскольку в них могут находиться паразиты. Но при этом от жарки на масле лучше отказаться,
-----
продуктах можно при минимальной тепловой обработке. Макароны или крупу рекомендуется 
    варить всего несколько минут. Кстати, классическая итальянская паста всегда должна быть чуть-чуть
-----
Правило № 1. Используйте для приготовления пищи несколько видов
    растительного масла. Это лучший источник необходимых организму жирных кислот.
    Очень хорошо, если на кухне постоянно будут несколько бутылочек с разными маслами:
-----


In [ ]:
!pip install gpt4all
# !pip install ctransformers
# пока неактуально

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q5_K_S.gguf
#!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf?download=true

Получаем ответ с помощью ЛЛМ

In [ ]:
question = "с чего начать, чтобы питаться правильно?"
# question = 'сколько было говядины'
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)
model_path = '/content/mistral-7b-instruct-v0.1.Q5_K_S.gguf'

llm = GPT4All(
    model=model_path,
    max_tokens = 256,
    callbacks=callback_manager,
    verbose=True
)
# это для лламы
template1 = """Ты - полезный ИИ ассистент для нашей организации.
Используй следующие фрагменты контекста (Context), чтобы ответить на вопрос в конце
(Question). Если ты не знаешь ответа, просто скажи, что не знаешь,
не пытайся придумывать ответ.
Сначала убедись, что прикрепленный текст имеет отношение к вопросу.
Если текст не имеет отношения к вопросу, просто скажи, что текст не имеет
отношения. Используй максимум три предложения. Дай ответ как можно более кратким.
Context: {context}
Question: {question}
Answer:"""  # лучше на английском, но у нас руссичи
# это для мистраля
template = f'"<s>[INST] {template1} [/INST] Model answer</s>[INST]"'


In [ ]:
# make chain
# где-то косяк. длительный поиск ответа и приходится делать кейбординтеррапт
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
# подключение к ллм
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    chain_type="stuff"
    # chain_type="map_reduce",
)
result = qa_chain({"query": question})
printAnswer(result['source_documents'])

NameError: name 'PromptTemplate' is not defined